In [12]:
from engaged.core import AzurePipeline as AP

In [16]:
def read_wav(df, meta):
    args = meta['read_wav']
    
    print "read_wav"
    import scipy.io.wavfile
    import pandas as pd
    
    sound = scipy.io.wavfile.read(args['in_filename'])
    df = pd.DataFrame(sound[1])
    return df, meta
    
def test2(df, meta):
#     args = meta['test2']
    
    print "test2"
    df /= 2
    return df, meta
    
def save(df, meta):
    args = meta['save']
    
    print "save"
    
    df.to_csv(args['out_filename'])
    
    return df, meta
    
    

In [17]:
meta = {'read_wav': {'in_filename': '../../../engaged_data/features/night.wav'},
        'save': {'out_filename': 'night.csv'}}

ap = AP.AzurePipeline(meta)

ap.apply(read_wav)
ap.apply(test2)
ap.apply(save)

test2
save
